# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y


0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Transform the features testing dataset
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [12]:
# Define the the number of inputs (features) to the model
inputs = X_train.shape[1]

# Review the number of features
print(f'Number of input features for X: {inputs}')

Number of input features for X: 11


In [27]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=11, activation='relu', input_dim=inputs))

# Add the second hidden layer
model.add(Dense(units=11, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=1, activation='sigmoid'))

In [28]:
# Display the Sequential model summary
# Display the Sequential model summary
model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 276 (1.08 KB)

 Trainable params: 276 (1.08 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [29]:
# Compile the Sequential model
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [30]:
# Fit the model using 50 epochs and the training data
model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 622us/step - accuracy: 0.5901 - loss: 0.6696 
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - accuracy: 0.6674 - loss: 0.6314
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.6941 - loss: 0.6019
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - accuracy: 0.7211 - loss: 0.5785
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - accuracy: 0.7304 - loss: 0.5475
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - accuracy: 0.7328 - loss: 0.5340
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.7487 - loss: 0.5200
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - accuracy: 0.7196 - loss: 0.5397
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.7443 - loss: 0.5145
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.7330 - loss: 0.5284
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.7227 - loss: 0.5297
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/ste

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [31]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - accuracy: 0.7608 - loss: 0.5030
Loss: 0.5187053084373474
Accuracy: 0.7425000071525574


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [34]:
# Set the model's file path
model_path = 'risk_assessment_model/student_loans.keras'

# Export your model to a keras file
model.save(model_path)
print(f"Model saved to {model_path}")


Model saved to risk_assessment_model/student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [37]:
# Set the model's file path
model_path = 'risk_assessment_model/student_loans.keras'

# Load the model to a new object
loan_risk_model = tf.keras.models.load_model(model_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [42]:
# Make predictions with the test data
predictions = loan_risk_model.predict(X_test_scaled)

# Round to Binary values for ease of reading
binary_predictions = (predictions > 0.5).astype(int)

# Display a sample of the predictions
print(binary_predictions[:20])


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step
[[1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]]


In [44]:
# Save the predictions to a DataFrame and round the predictions to binary results
# Create a DataFrame with the predictions
predictions_df = pd.DataFrame(binary_predictions, columns=['predictions'])

# Display the first few rows of the predictions DataFrame
predictions_df


,predictions
0,1
1,0
2,1
3,1
4,1
...,...
395,1
396,0
397,1
398,0


### Step 4: Display a classification report with the y test data and predictions

In [46]:
# Print the classification report with the y test data and predictions
report = classification_report(y_test, binary_predictions)
print(report)


              precision    recall  f1-score   support

           0       0.71      0.77      0.74       188
           1       0.78      0.72      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.75      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

### Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

To build a recommendation system for student loan options, we would need to collect the following data:

- Student's financial information: income, savings, existing debts
- Academic information: GPA, major, expected graduation date
- Loan details: interest rates, repayment terms, loan amounts
- Career prospects: expected salary after graduation, job placement rates
- Historical loan performance data: repayment rates, default rates

This data is relevant and appropriate because it provides a view of the student's financial situation, academic performance, and future earning potential. It allows the system to match students with loan options that they are likely to qualify for and be able to repay.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

The model would primarily use content-based filtering, with elements of context-based filtering when regarding career prospects. Content-based filtering is suitable because we're using specific attributes of the student (financial situation, academic performance) and the loans (interest rates, terms) to make recommendations. The system would match loan features to student profiles.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

Two real-world challenges to consider are:

a) Data privacy and security: Handling sensitive financial and personal information requires robust security measures to protect against data breaches and ensure compliance with regulations like FERPA in the US.

b) Ethical considerations: The system must avoid bias and ensure fair recommendations across all demographic groups. It should also consider the long-term financial impact on students, avoiding recommendations that could lead to excessive debt burdens.